# LittleMCMC Quickstart

LittleMCMC is a lightweight and performant implementation of HMC and NUTS in Python, spun out of the PyMC project. In this quickstart tutorial, we will walk through the main use case of LittleMCMC, and outline the various modules that may be of interest.

## Table of Contents

- [Who should use LittleMCMC?](#who-should-use-littlemcmc)
- [Sampling](#how-to-sample)
  - [Inspecting the Output of lmc.sample](#inspecting-the-output-of-lmc-sample)
- [Customizing the Default NUTS Sampler](#customizing-the-default-nuts-sampler)
- [Other Modules](#other-modules)

## Who should use LittleMCMC?

LittleMCMC is a fairly barebones library with a very niche use case. Most users will probably find that [PyMC3](https://github.com/pymc-devs/pymc3) will satisfy their needs, with better strength of support and quality of documentation.

There are two expected use cases for LittleMCMC. Firstly, if you:

1. Have model with only continuous parameters,
1. Are willing to manually transform all of your model's parameters to the unconstrained space (if necessary),
1. Have functions to compute the log probability of your model and its derivative, exposed through a Python callable,
1. And all you need is an implementation of HMC/NUTS (preferably in Python) to sample from the posterior,

then you should consider using LittleMCMC.

Secondly, if you want to run algorithmic experiments on HMC/NUTS (in Python), without having to develop around the heavy machinery that accompanies other probabilistic programming frameworks (like [PyMC3](https://github.com/pymc-devs/pymc3/), [TensorFlow Probability](https://github.com/tensorflow/probability/) or [Stan](https://github.com/stan-dev/stan)), then you should consider running your experiments in LittleMCMC.

## How to Sample

In [1]:
import numpy as np
import scipy
import littlemcmc as lmc

In [2]:
def logp_func(x, loc=0, scale=1):
    return np.log(scipy.stats.norm.pdf(x, loc=loc, scale=scale))


def dlogp_func(x, loc=0, scale=1):
    return -(x - loc) / scale


def logp_dlogp_func(x, loc=0, scale=1):
    return logp_func(x, loc=loc, scale=scale), dlogp_func(x, loc=loc, scale=scale)

In [3]:
trace, stats = lmc.sample(
    logp_dlogp_func=logp_dlogp_func,
    model_ndim=1,
    draws=1000,
    tune=500,
    progressbar=None,  # HTML progress bars don't render well in RST.
)

/home/george/littlemcmc/venv/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  


### Inspecting the Output of `lmc.sample`

In [4]:
trace

array([[ 0.91989736,  0.89910128,  0.88961585, ...,  1.31528136,
        -1.52068354, -0.61179308]])

In [5]:
trace.shape

(1, 4000)

In [6]:
stats

{'depth': array([2, 2, 1, ..., 2, 2, 1]),
 'step_size': array([1.15894778, 1.15894778, 1.15894778, ..., 1.50735926, 1.50735926,
        1.50735926]),
 'tune': array([False, False, False, ..., False, False, False]),
 'mean_tree_accept': array([0.75240493, 1.        , 1.        , ..., 0.96866415, 0.96184355,
        1.        ]),
 'step_size_bar': array([1.24138909, 1.24138909, 1.24138909, ..., 1.38460359, 1.38460359,
        1.38460359]),
 'tree_size': array([3., 3., 1., ..., 3., 3., 1.]),
 'diverging': array([False, False, False, ..., False, False, False]),
 'energy_error': array([ 0.13713966, -0.00798758, -0.00358263, ...,  0.09872431,
         0.1215682 , -0.40449202]),
 'energy': array([1.913862  , 1.5033014 , 1.48600433, ..., 1.82498413, 2.11625971,
        1.70961724]),
 'max_energy_error': array([ 0.45459613, -0.09927514, -0.00358263, ..., -0.25742807,
        -0.35807476, -0.40449202]),
 'model_logp': array([-1.34204411, -1.32313009, -1.31464672, ..., -1.78392107,
        -2.075

In [7]:
stats["diverging"].shape

(4000,)

## Customizing the Default NUTS Sampler

By default, `lmc.sample` samples using NUTS with sane defaults. These defaults can be override by either:

1. Passing keyword arguments from `lmc.NUTS` into `lmc.sample`, or
2. Constructing an `lmc.NUTS` sampler, and passing that to `lmc.sample`. This method also allows you to choose to other samplers, such as `lmc.HamiltonianMC`.

For example, suppose you want to increase the `target_accept` from the default `0.8` to `0.9`. The following two cells are equivalent:

In [8]:
trace, stats = lmc.sample(
    logp_dlogp_func=logp_dlogp_func,
    model_ndim=1,
    draws=1000,
    tune=500,
    progressbar=None,
    target_accept=0.9,
)

In [9]:
step = lmc.NUTS(logp_dlogp_func=logp_dlogp_func, model_ndim=1, target_accept=0.9)
trace, stats = lmc.sample(
    logp_dlogp_func=logp_dlogp_func,
    model_ndim=1,
    step=step,
    draws=1000,
    tune=500,
    progressbar=None,
)

For a list of keyword arguments that `lmc.NUTS` accepts, please refer to the [API reference for `lmc.NUTS`](https://littlemcmc.readthedocs.io/en/latest/generated/littlemcmc.NUTS.html#littlemcmc.NUTS).

## Other Modules

LittleMCMC exposes:

1. Two step methods (a.k.a. samplers): [`littlemcmc.HamiltonianMC` (Hamiltonian Monte Carlo)](https://littlemcmc.readthedocs.io/en/latest/generated/littlemcmc.HamiltonianMC.html#littlemcmc.HamiltonianMC) and the [`littlemcmc.NUTS` (No-U-Turn Sampler)](https://littlemcmc.readthedocs.io/en/latest/generated/littlemcmc.NUTS.html#littlemcmc.NUTS)
1. Various quadpotentials (a.k.a. mass matrices or inverse metrics) in [`littlemcmc.quadpotential`](https://littlemcmc.readthedocs.io/en/latest/api.html#quadpotentials-a-k-a-mass-matrices), along with mass matrix adaptation routines
1. Dual-averaging step size adaptation in [`littlemcmc.step_sizes`](https://littlemcmc.readthedocs.io/en/latest/generated/littlemcmc.step_sizes.DualAverageAdaptation.html#littlemcmc.step_sizes.DualAverageAdaptation)
1. A leapfrog integrator in [`littlemcmc.integration`](https://littlemcmc.readthedocs.io/en/latest/generated/littlemcmc.integration.CpuLeapfrogIntegrator.html#littlemcmc.integration.CpuLeapfrogIntegrator)

These modules should allow for easy experimentation with the sampler. Please refer to the [API Reference](https://littlemcmc.readthedocs.io/en/latest/api.html) for more information.